In [1]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from skimage import io, color, img_as_float, img_as_ubyte
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display

# Load and preprocess image
def load_image(filename):
    """Load image and convert to grayscale if needed"""
    img = io.imread(filename)
    if len(img.shape) > 2:  # Check if image is colored
        img = color.rgb2gray(img)
    return img

def histogram_equalize(image):
    """Apply histogram equalization to the image"""
    # Calculate histogram
    hist, bins = np.histogram(image.flatten(), 256, [0, 1])
    cdf = hist.cumsum()
    
    # Normalize cdf
    cdf_normalized = cdf * hist.max() / cdf.max()
    
    # Generate equalized image
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')
    
    return cdf[np.round(image * 255).astype(int)] / 255.0

def apply_dog_filter(img, sigma1, sigma2):
    """Apply Difference of Gaussians filter"""
    # Apply Gaussian filters
    gauss1 = ndimage.gaussian_filter(img, sigma=sigma1)
    gauss2 = ndimage.gaussian_filter(img, sigma=sigma2)
    
    # Calculate Difference of Gaussians
    dog = gauss1 - gauss2
    
    return gauss1, gauss2, dog

def detect_edges(dog_img, threshold):
    """Detect edges using the Sobel operator"""
    # Calculate Sobel gradients
    Gx = ndimage.sobel(dog_img, axis=1)
    Gy = ndimage.sobel(dog_img, axis=0)
    
    # Calculate magnitude and direction
    magnitude = np.sqrt(Gx**2 + Gy**2)
    angle = np.arctan2(Gy, Gx) * 180 / np.pi
    angle[angle < 0] += 180
    
    # Create edge mask based on threshold
    edge_mask = magnitude > threshold
    
    # Create directional edge map
    edge_map = np.zeros_like(dog_img)
    edge_map[edge_mask] = 1
    
    # Create directional edge classification
    direction = np.zeros_like(dog_img, dtype=np.uint8)
    direction[(angle >= 0) & (angle < 30) | (angle >= 150) & (angle <= 180)] = 1  # horizontal ('_')
    direction[(angle >= 30) & (angle < 60)] = 2  # diagonal ('/')
    direction[(angle >= 60) & (angle < 120)] = 3  # vertical ('|')
    direction[(angle >= 120) & (angle < 150)] = 4  # diagonal ('\\')
    
    # Only keep directions where there's an edge
    direction = direction * edge_mask
    
    return magnitude, direction, edge_map

# Interactive visualization for DoG parameter tuning
class DoGTuner:
    def __init__(self, image_path):
        self.img = load_image(image_path)
        self.eq_img = histogram_equalize(self.img)
        
        # Initial parameters
        self.sigma1 = 0.8
        self.sigma2 = 2.0
        self.threshold = 0.1
        
        # Calculate initial DoG
        self.update_dog()
        
        # Create UI
        self.create_widgets()
        
    def update_dog(self):
        """Update DoG calculation based on current parameters"""
        self.gauss1, self.gauss2, self.dog = apply_dog_filter(self.eq_img, self.sigma1, self.sigma2)
        self.magnitude, self.direction, self.edge_map = detect_edges(self.dog, self.threshold)
        
    def create_widgets(self):
        """Create interactive widgets"""
        # Create sliders
        self.sigma1_slider = widgets.FloatSlider(
            value=self.sigma1,
            min=0.1,
            max=5.0,
            step=0.1,
            description='Sigma 1:',
            continuous_update=False
        )
        
        self.sigma2_slider = widgets.FloatSlider(
            value=self.sigma2,
            min=0.1,
            max=10.0,
            step=0.1,
            description='Sigma 2:',
            continuous_update=False
        )
        
        self.threshold_slider = widgets.FloatSlider(
            value=self.threshold,
            min=0.01,
            max=1.0,
            step=0.01,
            description='Threshold:',
            continuous_update=False
        )
        
        # Create output selection
        self.output_selector = widgets.RadioButtons(
            options=['Original Image', 'Equalized Image', 'Gaussian 1', 'Gaussian 2', 'DoG', 'Edge Map'],
            value='DoG',
            description='Display:',
            disabled=False
        )
        
        # Add update callback
        self.sigma1_slider.observe(self.on_param_change, names='value')
        self.sigma2_slider.observe(self.on_param_change, names='value')
        self.threshold_slider.observe(self.on_param_change, names='value')
        self.output_selector.observe(self.on_display_change, names='value')
        
        # Create output area for the plot
        self.output = widgets.Output()
        
        # Display widgets
        display(widgets.VBox([
            widgets.HBox([self.sigma1_slider, self.sigma2_slider, self.threshold_slider]),
            self.output_selector,
            self.output
        ]))
        
        # Show initial plot
        self.update_plot()
        
    def on_param_change(self, change):
        """Callback for parameter changes"""
        self.sigma1 = self.sigma1_slider.value
        self.sigma2 = self.sigma2_slider.value
        self.threshold = self.threshold_slider.value
        
        self.update_dog()
        self.update_plot()
        
    def on_display_change(self, change):
        """Callback for display selection change"""
        self.update_plot()
        
    def update_plot(self):
        """Update the displayed plot"""
        with self.output:
            self.output.clear_output(wait=True)
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
            
            # Determine what to display in the first plot
            display_option = self.output_selector.value
            if display_option == 'Original Image':
                ax1.imshow(self.img, cmap='gray')
                title = 'Original Image'
            elif display_option == 'Equalized Image':
                ax1.imshow(self.eq_img, cmap='gray')
                title = 'Histogram Equalized Image'
            elif display_option == 'Gaussian 1':
                ax1.imshow(self.gauss1, cmap='gray')
                title = f'Gaussian Filter (σ={self.sigma1})'
            elif display_option == 'Gaussian 2':
                ax1.imshow(self.gauss2, cmap='gray')
                title = f'Gaussian Filter (σ={self.sigma2})'
            elif display_option == 'DoG':
                # Normalize DoG for better visualization
                dog_normalized = (self.dog - self.dog.min()) / (self.dog.max() - self.dog.min())
                ax1.imshow(dog_normalized, cmap='gray')
                title = f'DoG (σ1={self.sigma1}, σ2={self.sigma2})'
            else:  # Edge Map
                ax1.imshow(self.edge_map, cmap='gray')
                title = f'Edge Map (threshold={self.threshold})'
                
            ax1.set_title(title)
            ax1.axis('off')
            
            # Always show ASCII-like edge visualization in second plot
            # Create a visualization similar to what you'd get in ASCII art
            edge_viz = np.zeros_like(self.img)
            
            # Set different values based on direction for visualization
            edge_viz[self.direction == 1] = 0.25  # horizontal
            edge_viz[self.direction == 2] = 0.5   # diagonal /
            edge_viz[self.direction == 3] = 0.75  # vertical
            edge_viz[self.direction == 4] = 1.0   # diagonal \
            
            ax2.imshow(edge_viz, cmap='gray')
            ax2.set_title(f'Edge Direction Visualization (threshold={self.threshold})')
            ax2.axis('off')
            
            plt.tight_layout()
            plt.show()
            
            # Print the parameters for reference
            print(f"Current parameters: σ1={self.sigma1}, σ2={self.sigma2}, threshold={self.threshold}")
            print(f"DoG range: min={self.dog.min():.4f}, max={self.dog.max():.4f}")
            
            # Convert values to match your CUDA code settings 
            # (your code uses 0.8 and 2.0 for sigmas, and threshold value of 100)
            cuda_threshold = self.threshold * 255
            print("\nValues for CUDA code:")
            print(f"gaussianBlur<<<gridSize, blockSize>>>(d_eqImg, d_gauss1, width, height, {self.sigma1}f);")
            print(f"gaussianBlur<<<gridSize, blockSize>>>(d_eqImg, d_gauss2, width, height, {self.sigma2}f);")
            print(f"asciiEdgeKernel<<<gridSize, blockSize>>>(d_dog, d_eqImg, d_ascii, width, height, {int(cuda_threshold)}, dsWidth);")

# Example usage - replace 'your_image.jpg' with your image path
# tuner = DoGTuner('your_image.jpg')

# To use this notebook, run the cell above, then execute:
# tuner = DoGTuner('path/to/your/image.jpg')

def test_with_vertical_gradient():
    """Create a test image with vertical gradient for easy visualization"""
    # Create vertical gradient test image
    x = np.linspace(0, 1, 512)
    y = np.linspace(0, 1, 512)
    X, Y = np.meshgrid(x, y)
    test_img = Y  # Vertical gradient
    
    # Add some features for edge detection
    test_img[200:300, 100:400] = 0.8
    test_img[150:450, 250:270] = 0.2
    
    # Save and return
    io.imsave('test_gradient.png', img_as_ubyte(test_img))
    return 'test_gradient.png'

# Uncomment to create and use a test gradient image
# test_img_path = test_with_vertical_gradient()
# tuner = DoGTuner(test_img_path)

def apply_full_ascii_conversion(image_path, sigma1=0.8, sigma2=2.0, threshold=0.1, downsample=2):
    """Apply full ASCII art conversion pipeline similar to your CUDA code"""
    # Load image
    img = load_image(image_path)
    
    # Histogram equalization
    eq_img = histogram_equalize(img)
    
    # Apply DoG filter
    _, _, dog = apply_dog_filter(eq_img, sigma1, sigma2)
    
    # Detect edges
    magnitude, direction, _ = detect_edges(dog, threshold)
    
    # Downsample and create ASCII representation
    h, w = img.shape
    ds_h, ds_w = h // downsample, w // downsample
    
    # Character set similar to your CUDA code
    char_set = " .,-~:;=!*#$@█"
    
    # Create ASCII art
    ascii_art = []
    for y in range(0, h, downsample):
        line = []
        for x in range(0, w, downsample):
            # Get block for downsampling
            if y+downsample <= h and x+downsample <= w:
                block = magnitude[y:y+downsample, x:x+downsample]
                dir_block = direction[y:y+downsample, x:x+downsample]
                
                # Edge detection similar to your CUDA code
                if np.max(block) > threshold:
                    # Get predominant direction
                    dirs = dir_block[dir_block > 0]
                    if len(dirs) > 0:
                        dir_val = np.bincount(dirs.flatten()).argmax()
                        if dir_val == 1:
                            ch = '_'
                        elif dir_val == 2:
                            ch = '/'
                        elif dir_val == 3:
                            ch = '|'
                        else:
                            ch = '\\'
                    else:
                        # Fallback to intensity if no direction
                        val = eq_img[y, x]
                        ramp_index = int(val * (len(char_set) - 1))
                        ch = char_set[ramp_index]
                else:
                    # Use intensity-based character
                    val = eq_img[y, x]
                    ramp_index = int(val * (len(char_set) - 1))
                    ch = char_set[ramp_index]
                line.append(ch)
        ascii_art.append(''.join(line))
    
    # Display and save result
    print('\n'.join(ascii_art))
    
    with open('ascii_output.txt', 'w') as f:
        for line in ascii_art:
            f.write(line + line + '\n')  # Double characters like in CUDA code
    
    return ascii_art

# Example usage
# ascii_result = apply_full_ascii_conversion('your_image.jpg', 0.8, 2.0, 0.1)

In [3]:
tuner = DoGTuner('b_ascii_at/input.png')